In [ ]:
# ============ IMPORTS ============
import pandas as pd
import numpy as np
import talib
import os
from sklearn.preprocessing import MinMaxScaler
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============ LOAD CACHED DATA ============
cache_file = 'data_cache/bitcoin_data.pkl'

if not os.path.exists(cache_file):
    raise FileNotFoundError("Cache not found. Run the 'bitcoin_data_fetch.ipynb' Jupyter notebook.")

cache = pd.read_pickle(cache_file)
bitcoin_historical_data = cache['bitcoin_data']
bitcoin_fgi_data = cache['fgi_data']
start_date = cache['start_date']
end_date = cache['end_date']

print(f" Loaded from {cache['cache_date']}")

# ============ CALCULATE INDICATORS ============
btc_close = bitcoin_historical_data[('Close', 'BTC-USD')].values

bitcoin_historical_data['RSI'] = talib.RSI(btc_close, timeperiod=14)
macd_line, macd_signal, macd_hist = talib.MACD(btc_close, fastperiod=12, slowperiod=26, signalperiod=9)
bitcoin_historical_data['MACD'] = macd_hist
bitcoin_historical_data['MACD_Signal'] = macd_signal
bitcoin_historical_data['Fear_and_Greed_Index'] = bitcoin_fgi_data.set_index('Date')['value']

# ============ COLOR MAPPING ============
indicator_color_mapping = {
    ('Close', 'BTC-USD'): 'blue',
    'RSI': 'red',
    'MACD': 'green',
    'MACD_Signal': 'orange',
    'Fear_and_Greed_Index': 'purple'
}

# ============ NORMALISE DATA ============
# Store both absolute (original) and normalised (0-1 scaled) values
data_storage = {'absolute_values': {}, 'normalised_values': {}}
scaler = MinMaxScaler()

indicators = [('Close', 'BTC-USD'), 'RSI', 'MACD', 'MACD_Signal', 'Fear_and_Greed_Index']

for indicator in indicators:
    # Store absolute values
    data_storage['absolute_values'][indicator] = bitcoin_historical_data[indicator].values
    
    # Normalise only valid (non-NaN) values
    valid_mask = bitcoin_historical_data[[indicator]].notna().values.flatten()
    normalised = np.full(len(bitcoin_historical_data), np.nan)
    
    if valid_mask.any():
        valid_data = bitcoin_historical_data.loc[valid_mask, [indicator]].values
        normalised[valid_mask] = scaler.fit_transform(valid_data).flatten()
    
    data_storage['normalised_values'][indicator] = normalised

 Loaded from 2025-11-20 12:58


In [2]:
# ============ DASH APP FOR JUPYTER NOTEBOOK ============
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============ CONFIGURATION ============
app = dash.Dash(__name__)

# Bitcoin halving event dates
halving_dates = [
    pd.to_datetime('2012-11-28'),  # 50 → 25 BTC
    pd.to_datetime('2016-07-09'),  # 25 → 12.5 BTC
    pd.to_datetime('2020-05-11'),  # 12.5 → 6.25 BTC
    pd.to_datetime('2024-04-19'),  # 6.25 → 3.125 BTC
]

# Indicator dropdown options
indicator_options = [
    {'label': 'None', 'value': 'None'},
    {'label': 'RSI (Absolute)', 'value': 'RSI_abs'},
    {'label': 'RSI (Normalised)', 'value': 'RSI_norm'},
    {'label': 'MACD (Absolute)', 'value': 'MACD_abs'},
    {'label': 'MACD (Normalised)', 'value': 'MACD_norm'},
    {'label': 'MACD Signal (Absolute)', 'value': 'MACD_Signal_abs'},
    {'label': 'MACD Signal (Normalised)', 'value': 'MACD_Signal_norm'},
    {'label': 'Fear & Greed (Absolute)', 'value': 'Fear_and_Greed_Index_abs'},
    {'label': 'Fear & Greed (Normalised)', 'value': 'Fear_and_Greed_Index_norm'},
]

# ============ APP LAYOUT ============
app.layout = html.Div([
    html.H2("Bitcoin Price Cycles with Technical Indicators", 
            style={'textAlign': 'center'}),
    
    # Control panel
    html.Div([
        html.Label("BTC Price Display:"),
        dcc.RadioItems(
            id='btc-scale',
            options=[{'label': ' Absolute (USD)', 'value': 'abs'},
                    {'label': ' Normalised (0-1)', 'value': 'norm'}],
            value='abs',
            inline=True,
            style={'marginLeft': '10px'}
        ),
        html.Label("Y-Axis Scale:", style={'marginLeft': '30px'}),
        dcc.RadioItems(
            id='y-scale',
            options=[{'label': ' Linear', 'value': 'linear'},
                    {'label': ' Logarithmic', 'value': 'log'}],
            value='log',
            inline=True,
            style={'marginLeft': '10px'}
        )
    ], style={'textAlign': 'center', 'padding': '10px', 'backgroundColor': '#f0f0f0'}),
    
    # Indicator selectors
    html.Div([
        html.Div([
            html.Label(f"Indicator Row {i}:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id=f'indicator-row-{i}',
                options=indicator_options,
                value='None',
                clearable=False,
                style={'margin': 'auto', 'width': '300px'}
            )
        ], style={'textAlign': 'center', 'padding': '5px'})
        for i in [2, 3, 4]
    ]),
    
    dcc.Graph(id='btc-indicator-graph', style={'height': '1400px'})
])

# ============ HELPER FUNCTIONS ============
def add_btc_trace(fig, data, scale):
    """Add Bitcoin price trace to row 1"""
    if scale == 'abs':
        y_data = data[('Close', 'BTC-USD')]
        name, template = 'BTC Price', '<b>BTC Price</b><br>$%{y:,.2f}<extra></extra>'
        line = dict(color='blue', width=2)
    else:
        y_data = data_storage['normalised_values'][('Close', 'BTC-USD')]
        name, template = 'BTC Price (Norm)', '<b>BTC Price (Norm)</b><br>%{y:.4f}<extra></extra>'
        line = dict(color='blue', width=2, dash='dash')
    
    fig.add_trace(go.Scatter(
        x=data.index, y=y_data, name=name,
        line=line, hovertemplate=template
    ), row=1, col=1)

def add_halving_lines(fig):
    """Add vertical lines and annotations for Bitcoin halving events"""
    for halving_date in halving_dates:
        if start_date <= halving_date <= end_date:
            for row_num in [1, 2, 3, 4]:
                fig.add_shape(
                    type="line", x0=halving_date, x1=halving_date, y0=0, y1=1,
                    xref=f'x{row_num}' if row_num > 1 else 'x',
                    yref=f'y{row_num} domain' if row_num > 1 else 'y domain',
                    line=dict(color="red", width=2, dash="dash"), opacity=0.6
                )
            # Annotation only on top plot
            fig.add_annotation(
                x=halving_date, y=1.0, xref='x', yref='y domain',
                text="Halving", showarrow=False, yanchor='bottom',
                font=dict(size=10, color='red')
            )

def add_indicator_or_zeroes(fig, row_num, indicator_value):
    """Add indicator trace or invisible zeroes for rangeslider anchor"""
    if indicator_value == 'None':
        # Invisible dummy trace for empty rows
        fig.add_trace(go.Scatter(
            x=bitcoin_historical_data.index,
            y=[0] * len(bitcoin_historical_data),
            mode='lines', line=dict(width=0),
            showlegend=False, hoverinfo='skip'
        ), row=row_num, col=1)
    else:
        # Parse indicator name and type
        indicator, ind_type = indicator_value.rsplit('_', 1)
        
        # Get data and styling
        if ind_type == 'abs':
            y_data = data_storage['absolute_values'][indicator]
            line = dict(color=indicator_color_mapping[indicator], width=2)
        else:
            y_data = data_storage['normalised_values'][indicator]
            line = dict(color=indicator_color_mapping[indicator], width=2, dash='dot')
        
        # Add trace
        fig.add_trace(go.Scatter(
            x=bitcoin_historical_data.index, y=y_data,
            name=f'{indicator} ({"Abs" if ind_type == "abs" else "Norm"})',
            line=line,
            hovertemplate=f'<b>{indicator}</b><br>%{{y:.4f}}<extra></extra>'
        ), row=row_num, col=1)

# ============ MAIN CALLBACK ============
@app.callback(
    Output('btc-indicator-graph', 'figure'),
    [Input('btc-scale', 'value'),
     Input('y-scale', 'value'),
     Input('indicator-row-2', 'value'),
     Input('indicator-row-3', 'value'),
     Input('indicator-row-4', 'value')]
)
def update_figure(btc_scale, y_scale, row2_val, row3_val, row4_val):
    # Create subplot structure
    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        row_heights=[0.4, 0.2, 0.2, 0.2],
        subplot_titles=('Bitcoin Price (with Halving Events)',
                       'Indicator Row 2', 'Indicator Row 3', 'Indicator Row 4')
    )
    
    # Add all traces
    add_btc_trace(fig, bitcoin_historical_data, btc_scale)
    add_halving_lines(fig)
    
    for row_num, indicator_val in zip([2, 3, 4], [row2_val, row3_val, row4_val]):
        add_indicator_or_zeroes(fig, row_num, indicator_val)
    
    # Configure y-axes
    fig.update_yaxes(title_text="<b>BTC Price (USD)</b>", type=y_scale, row=1, col=1)

    # For indicator rows, force linear scale with proper range
    for row_num, indicator_val in zip([2, 3, 4], [row2_val, row3_val, row4_val]):
        if indicator_val != 'None':
            indicator, ind_type = indicator_val.rsplit('_', 1)
            
            if ind_type == 'abs':
                y_data = data_storage['absolute_values'][indicator]
                y_min = y_data.min() * 0.9 if y_data.min() > 0 else y_data.min() * 1.1
                y_max = y_data.max() * 1.1
            else:
                y_min, y_max = -0.05, 1.05
            
            fig.update_yaxes(
                title_text=f"<b>{indicator}</b>",
                type='linear',
                range=[y_min, y_max],
                row=row_num,
                col=1
            )
        else:
            fig.update_yaxes(
                title_text="<b>Indicator Value</b>",
                type='linear',
                row=row_num,
                col=1
            )
    
    # Configure x-axes
    fig.update_xaxes(showline=True, showticklabels=True, ticks='outside', tickangle=0)
    fig.update_xaxes(title_text="<b>Date</b>", row=4, col=1)
    
    # Add rangeslider and range selector to bottom subplot
    fig.update_xaxes(
        rangeslider=dict(visible=True, thickness=0.05,
                        bgcolor='rgba(200, 200, 200, 1)',
                        bordercolor='#888', borderwidth=2),
        rangeselector=dict(
            buttons=[
                dict(count=1, label='1M', step='month', stepmode='backward'),
                dict(count=6, label='6M', step='month', stepmode='backward'),
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=2, label='2Y', step='year', stepmode='backward'),
                dict(step='all', label='All')
            ],
            x=0.0, xanchor='left', yanchor='top'
        ),
        row=4, col=1
    )
    
    # Final layout configuration
    fig.update_layout(
        hovermode='x unified',
        height=1200,
        template='plotly_white',
        showlegend=True,
        xaxis={'matches': 'x4'},   # Link all x-axes to bottom subplot
        xaxis2={'matches': 'x4'},
        xaxis3={'matches': 'x4'}
    )
    
    return fig

# ============ RUN APP ============
if __name__ == '__main__':
    app.run(jupyter_mode='tab')

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>